In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
import transformers
import load_data
import torch
from torch import nn
from transformers import BertModel, BertTokenizer, CamembertModel, CamembertTokenizer
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import f1_score
import numpy as np
from sklearn.metrics import confusion_matrix
import os
import gc
from pydub import AudioSegment
import os
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu') # GPU ou CPU
print(torch.cuda.is_available())
import librosa

In [ ]:
import os
import gc
import numpy as np
import pandas as pd
import librosa
from pydub import AudioSegment
from sklearn.model_selection import train_test_split
from transformers import CamembertTokenizer
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.preprocessing import LabelEncoder

In [ ]:
# ------------------- CHARGEMENT ET PRÉTRAITEMENT DES DONNÉES TEXTUELLES -------------------

# Remplacez ceci par votre propre fonction de chargement des données
transcr_path = 'paco-cheese/transcr'
data = load_data.load_all_ipus(folder_path=transcr_path, load_words=True) #fonction donnée par le prof

In [ ]:
# Conversion des données en DataFrame
df = pd.DataFrame(data)

# Extraction de l'information de changement de locuteur
y = [0 if i == 0 else 1 for i in range(len(df)) if df['speaker'][i] != df['speaker'][i - 1]]

In [ ]:
# ------------------- EXTRACTION DES CARACTÉRISTIQUES AUDIO -------------------
audio_files_path = 'paco-cheese/audio/2_channels/'

# Cette fonction prend une entrée de dyade et renvoie le chemin du fichier audio correspondant
def find_audio_file(dyad,first_speaker):
    dyad=dyad.split('\\')[1]
    #first_speaker=str(first_speaker)
    if isinstance(first_speaker, float):
        first_speaker="NA"
        print(first_speaker)
    second_speaker=dyad.replace(first_speaker,"")
    #if "LS" in dyad:
    #    print("trouvé")
    # Trouver le fichier audio qui contient l'identifiant de la dyade
    for file_name in os.listdir(audio_files_path):
        #if("LS" in dyad):
            #print(first_speaker)
            #print(second_speaker)
            #print(file_name)
        if first_speaker in file_name and second_speaker in file_name:
            return os.path.join(audio_files_path, file_name)
    return None

# Cette fonction extrait les segments audio en utilisant les informations de la dyade et les timestamps
def extract_audio_segments(df):
    audio_segments = []
    audio_file_path = ""
    nombre_boucle=0
    for index, row in df.iterrows():
        nombre_boucle+=1
        first_speaker=str(row['speaker'])
        if isinstance(row['speaker'], float):
            first_speaker="NA"
            
            
        if first_speaker not in audio_file_path:
            # Si le fichier audio n'a pas encore été chargé, chargez-le
            audio = None
            gc.collect()
            audio_file_path = find_audio_file(row['dyad'],first_speaker)
            if audio_file_path is None:
                print("Audio file not found for dyad {}".format(row['dyad']))
                audio_file_path = ""
                continue
            audio = AudioSegment.from_file(audio_file_path)
            #print(index)
            print(audio_file_path)  
        if audio_file_path!="":
            start_ms = int(row['start_ipu'] * 1000)
            end_ms = int(row['stop_ipu'] * 1000)
            segment = audio[start_ms:end_ms]
            audio_segments.append(segment)
            # Vous pouvez également enregistrer le segment si nécessaire
            # segment.export('segment_{}.wav'.format(index), format='wav')
        if audio_file_path=="":
            print("trouvé")
            print(audio_file_path)
    return audio_segments,nombre_boucle

# Utilisez la fonction pour votre DataFrame
audio_segments,nombre_boucle = extract_audio_segments(data)

def extract_features(audio_segment):
    # Convert PyDub audio segment to numpy array
    samples = np.array(audio_segment.get_array_of_samples())

    # Normalize the audio samples to floating-point values
    if audio_segment.sample_width == 2:
        samples = samples.astype(np.float32) / 32768
    elif audio_segment.sample_width == 4:
        samples = samples.astype(np.float32) / 2147483648

    # Use librosa to extract MFCCs
    mfccs = librosa.feature.mfcc(y=samples, sr=audio_segment.frame_rate, n_mfcc=13)
    
    # Average the MFCCs over time
    mfccs_mean = np.mean(mfccs, axis=1)
    return mfccs_mean

In [ ]:
# Extraction des caractéristiques MFCC pour tous les segments audio
audio_segments_small=audio_segments[:1000]
# X_audio = np.array([extract_features(segment) for segment in audio_segments])

In [ ]:
X_audio=X[:1000]

In [ ]:
# ------------------- FUSION DES CARACTÉRISTIQUES AUDIO ET TEXTUELLES -------------------

# Prétraitement des données textuelles
text_features = df['text_words'].fillna('[UNK]')

# Utilisation du tokenizer CamemBERT
tokenizer = CamembertTokenizer.from_pretrained('camembert-base')
text_features = text_features.apply(lambda x: tokenizer.encode(x, add_special_tokens=True, padding='max_length', truncation=True))

# Conversion des caractéristiques textuelles en tensor
X_text = torch.tensor(np.array(text_features.tolist()))

# Fusion des caractéristiques audio et textuelles
X_combined = torch.cat((torch.tensor(X_audio), X_text), dim=1)

In [ ]:
# Division en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.3, random_state=42)

In [ ]:
# ------------------- CONSTRUCTION ET ENTRAÎNEMENT DU MODÈLE EN PYTORCH -------------------


# Définition du dataset
class CombinedDataset(Dataset):
    def __init__(self, features, labels):
        """
        Initialisation du dataset.
        :param features: Les caractéristiques fusionnées (audio et texte).
        :param labels: Les étiquettes correspondantes.
        """
        self.features = features
        self.labels = labels

    def __len__(self):
        """
        Retourne la longueur totale du dataset.
        """
        return len(self.features)

    def __getitem__(self, idx):
        """
        Récupère l'échantillon de données et son étiquette à l'index spécifié.
        :param idx: Index de l'échantillon à récupérer.
        :return: Tuple (échantillon, étiquette).
        """
        return self.features[idx], self.labels[idx]


# Création des DataLoaders
train_dataset = CombinedDataset(X_train, y_train)
test_dataset = CombinedDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Définition du modèle de réseau de neurones
class EarlyFusionModel(nn.Module):
    # Votre architecture de modèle
    pass

# Entraînement du modèle
model = EarlyFusionModel()

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import CamembertTokenizer
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Chargement des données textuelles et extraction des caractéristiques
# Assurez-vous que la variable 'data' est chargée comme dans votre script précédent
text_features = data['text_words']

# Remplacer les valeurs NaN par un placeholder
text_features = text_features.fillna('[UNK]')

# Utiliser CamemBERT tokenizer
tokenizer = CamembertTokenizer.from_pretrained('camembert-base')

def tokenize_and_pad(text, max_len=256):
    return tokenizer.encode(text, max_length=max_len, padding='max_length', truncation=True)

# Appliquer le tokenizer à toutes les données textuelles
text_features = text_features.apply(lambda x: tokenize_and_pad(str(x)))

# Assurez-vous que 'X' est votre matrice de caractéristiques audio extraites
audio_features = torch.tensor(X)

# Convertir les caractéristiques textuelles en tensor
text_features = torch.tensor(np.array(text_features.tolist()))

# Fusionner les caractéristiques audio et textuelles
combined_features = torch.cat((audio_features, text_features), dim=1)

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(combined_features, y, test_size=0.3, random_state=42)

# Créer des datasets et des dataloaders
class CombinedDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = torch.tensor(labels)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

train_dataset = CombinedDataset(X_train, y_train)
test_dataset = CombinedDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Définition du modèle de réseau de neurones
class EarlyFusionModel(nn.Module):
    def __init__(self, input_size):
        super(EarlyFusionModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

model = EarlyFusionModel(X_train.shape[1])

# Entraînement du modèle
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(10):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels.float())
        loss.backward()
        optimizer.step()

# Évaluation du modèle
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        predicted = (outputs.squeeze() > 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Accuracy: {accuracy}')
